# ResNet
CNN: 흑백 사진 <br>
컬러 데이터셋 사용 :ResNet

- 신경망을 깊게 쌓으면 오히려 성능이 나빠짐으로 이의 문제 해결, DenseNet등의 파생모델에 영향
- 컨볼루션 층의 출력에 전의 전 계층에 쓰였던 입력을 더함으로써 특징이 유실되지 않도록 함

RESNET <br>
네트워크를 작은 블록인 **Residual 블록(=BasicBlock 파이토치 모듈로 정의)** 로 나눔
* Batch Normalization -> 성능은 Dropout과 같음 : 중간에 더해주기 때문에 표준화 필요
    - Dropout: 학습 중 데이터 일부를 제외 시키고 과적합 막음
    - BN: 신경망 내부, 데이터에 직접 영향<br>
    <br>
**Residual Block 구조** 
<br> <br>
inplanes(x) => [(conv1->bn1) : inplanes -> planes] => ReLU => [(conv1->bn1) : planes -> planes]
          =>                                                                     => + => ReLU -> out

In [ ]:
import torch
import torch.nn as nn
import torch.optim  as optim
import torch.nn.functional as F
from torchvision import transforms, datasets, models

In [3]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [4]:
EPOCHS = 300
BATCH_SIZE = 128

CIFAR-10 : 10개의 이미지 분류
- R,G,B: 8bit씩 (0~225) 3채널
- PNG는 투명도까지 4종류의 채널

In [5]:
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./.data',
                    train = True,
                    download = True,
                    transform = transforms.Compose([
                        transforms.RandomCrop(32, padding = 4),
                        transforms.RandomHorizontalFlip(),
                        transforms.ToTensor(),
                        transforms.Normalize((0.5, 0.5, 0.5),
                                            (0.5,0.5,0.5))])),
        batch_size = BATCH_SIZE, shuffle = True
)
#과적합 방지하기 위하여 RandomCrop, RandomHorizontalFlip 사용

test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./.data',
                    train = False,
                    transform = transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.5, 0.5, 0.5),
                                            (0.5,0.5,0.5))])),
        batch_size = BATCH_SIZE, shuffle = True
)

Extracting ./.data\cifar-10-python.tar.gz to ./.data


#### Residual 블록의 출력에 입력이었던 x를 더함으로써 모델을 훨씬 깊게 설계할 수 있도록 함

**배치 정규화**
- 학습률을 너무 높게 잡으면 기울기가 소실되거나 발산하는 증상을 예방하여 학습과정을 안정화하는 방법
- 각 계층에 들어가는 입력을 평균과 분산으로 정규화 함으로써 학습을 효율적으로 만들어줌
    - 드롭아웃과 같은 효과를 냄

In [23]:
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride = 1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size = 3, stride = stride, padding = 1, bias =False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size = 3, stride = 1, padding = 1, bias = False)
        self.bn2 = nn.BatchNorm2d(planes)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size = 1, stride = stride, bias = False),
                nn.BatchNorm2d(planes)
            )
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn1(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [24]:
class ResNet(nn.Module):
    def __init__(self, num_classes =10):
        super(ResNet, self).__init__()
        self.in_planes = 16 #layer 1층이 입력 받는 채널의 수
        self.conv1 = nn.Conv2d(3,16, kernel_size =3, stride = 1, padding = 1, bias = False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(16,2, stride = 1)
        self.layer2 = self._make_layer(32,2, stride = 2)
        self.layer3 = self._make_layer(64,2, stride = 2)
        self.linear = nn.Linear(64, num_classes)
        
    def _make_layer(self, planes, num_blocks, stride):
        stride = [stride] + [1]*(num_blocks - 1)
        layers = []
        for stride in stride:
            layers.append(BasicBlock(self.in_planes, planes, stride))
            self.in_planes = planes
        return nn.Sequential(*layers)
    
    def forward(self,x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

- conv [3채널 -> 16채널]
- Layer1:
    - BasicBlock[16채널->16채널]
    - BasicBlock[16채널->16채널]
- Layer2:
    - BasicBlock[16채널->32채널] + shortcut 모듈
    - BasicBlock[32채널->32채널]
- Layer3:
    - BasicBlock[32채널->64채널] + shortcut 모듈
    - BasicBlock[64채널->64채널]
- linear [64채널 -> 10채널]

**증폭되는 BasicBlock은 shortcut 모듈도 같이 가지게 됨. shortcut 모듈은 이전 입력을 중간층에 더해주어 이미지 맥락이 보존될 수 있도록 하는 것**
- BasicBlock 클래스는 ResNet의 self._make_layer() 함수를 통해 하나의 모듈로 객체화 되어 우리가 만들 ResNet 모델의 중요층이다.
- self._make_layer() 함수: 파이토치의 nn.Sequential 도구로 여러 BasicBlock을 모듈 하나로 묶어줌

**학습률 감소 기법 사용**
학습이 진행하면서 최적화 함수의 학습률을 점점 낮춰서 더 정교하게 최적화함

In [25]:
model = ResNet().to(DEVICE)
optimizer= optim.SGD(model.parameters(), lr = 0.1, momentum =0.9, weight_decay = 0.0005)
# EPOCH 마다 호출되며 step_size 를 50으로 지정해주어 50번 호출될때 학습률에 0.1(gamma 값)을 곱한다. 0.1로 시작한 학습률은 50 EPoch 이후에 0.01로 낮아짐
scheduler  = optim.lr_scheduler.StepLR(optimizer, step_size = 50, gamma = 0.1) 

In [26]:
print(model)

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=

In [31]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 200 == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss:{:.6f}"
                 .format(epoch, batch_idx * len(data),
                        len(train_loader.dataset),
                        100.*batch_idx / len(train_loader),
                        loss.item()))

In [32]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            #배치 오차률 합산
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            
            #가장 높은 값을 가진 인덱스가 바로 예측값
            pred = output.max(1, keepdim =True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [33]:
for epoch in range(1, EPOCHS+ 1):
    scheduler.step()
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = test(model, test_loader)
    
    print("[{}] Test Loss:{:.4f}, Accuracy: {:.2f}%".format(epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/50000 (0%)]	Loss:2.322820


KeyboardInterrupt: 